In [1]:
import pandas as pd
import copy

In [2]:
trains_dataframe = pd.read_csv("dataset/Regularities_by_liaisons_Trains_France.csv")

In [3]:
print(trains_dataframe.isnull().values.any())


True


In [4]:
trains_dataframe = trains_dataframe.drop(columns=['Comment (optional) delays at departure','Comment (optional) delays on arrival'])

In [5]:
visualisation = trains_dataframe[0:11].style.hide(['Year', 'Month', 'Departure station', 'Arrival station',
                                             'Number of expected circulations',
                                             'Number of cancelled trains', 'Number of late trains at departure',
                                             'Average delay of late departing trains (min)',
                                             'Average delay of all departing trains (min)',
                                             'Average delay of late arriving trains (min)',
                                             '% trains late due to external causes (weather, obstacles, suspicious packages, malevolence, social movements, etc.)',
                                             '% trains late due to railway infrastructure (maintenance, works)',
                                             '% trains late due to traffic management (rail line traffic, network interactions)',
                                             '% trains late due to rolling stock',
                                             '% trains late due to station management and reuse of material',
                                             '% trains late due to passenger traffic (affluence, PSH management, connections)',
                                             'Number of late trains > 15min',
                                             'Number of late trains > 30min', 'Number of late trains > 60min', 'Period',
                                             'Delay due to external causes', 'Delay due to railway infrastructure',
                                             'Delay due to traffic management', 'Delay due to rolling stock',
                                             'Delay due to station management and reuse of material',
                                             'Delay due to travellers taken into account'], axis="columns")

visualisation.highlight_max(color='green').highlight_min(color='red')

,Average travel time (min),Number of trains late on arrival,Average delay of all arriving trains (min),Average train delay > 15min
0,131.914980,41.000000,5.233333,32.965873
1,175.611570,47.000000,6.123967,32.057143
2,62.395349,91.000000,4.430194,42.367241
3,172.421053,15.000000,3.812135,27.620833
4,67.310000,73.000000,4.176208,37.658333
5,266.594737,28.000000,8.422105,47.343452
6,265.262887,29.000000,8.407474,45.610920
7,85.399381,52.000000,5.523633,60.874405
8,46.305556,36.000000,5.154244,47.841667
9,67.000000,30.000000,3.633051,35.613889


In [6]:
del trains_dataframe
trains_dataframe = pd.read_csv("dataset/Regularities_by_liaisons_Trains_France.csv")

trains_dataframe = trains_dataframe.drop(columns=['Comment (optional) delays at departure','Comment (optional) delays on arrival'])
trains_dataframe = trains_dataframe.dropna()
print(trains_dataframe.isnull().values.any())
trains_dataframe_numerical = trains_dataframe.drop(columns=['Year', 'Month', 'Departure station', 'Arrival station'])

False


In [8]:
def standartizatinator(input_column):
    values = []
    probability = []
    length = len(input_column)
    for element in input_column:
        if element in values:
            probability[values.index(element)]+=1
        else:
            probability.append(1)
            values.append(element)
    i = 0
    mu = 0
    while i < len(values):
        
        mu+=probability[i]*values[i]/length
        i+=1
    
    avg = sum(input_column)/length
    i = 0
    some_weird_letter = 0
    for i in input_column:
        some_weird_letter += i-avg
    
    some_weird_letter = (abs(some_weird_letter/(len(input_column)-1)))**(1/2)
    i = 0
    while i < len(input_column):
        input_column[i] = (input_column[i]-mu)/some_weird_letter
        i+=1
    return input_column

In [10]:
trains_dataframe_numerical_standart = trains_dataframe_numerical
trains_dataframe_numerical_standart['Number of expected circulations'] = standartizatinator(trains_dataframe_numerical['Number of expected circulations'])

C:\Users\misha\AppData\Local\Temp\ipykernel_33192\242169109.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_column[i] = (input_column[i]-mu)/some_weird_letter


KeyError: 12